In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader 

import torchvision
import torchvision.transforms as transforms

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
trans=transforms.Compose([transforms.ToTensor()])   # transforms.Compose(): 여러 개의 이미지 변환(transform)을 하나로 묶어서 적용할 수 있도록 도와주는 함수 
                                                    # 여러 개의 변환을 순차적으로 진행하기 때문에 []안에 넣는다.
                                                    # transforms.ToTensor(): 이 변환을 적용하면 이미지를 [0, 1] 범위의 실수형 텐서로 변환
train_data=torchvision.datasets.ImageFolder(root='./custom_data/train_data',transform=trans)

In [4]:
data_loader=DataLoader(dataset=train_data, batch_size=8, shuffle=True, num_workers=2)   #num_workers는 돌아가는 cpu의 개수를 의미.

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(3,6,5),           # input_channel, output_channel, kernel_size순. stride는 기본값이 1임. padding 도 기본값이 0임.
            nn.ReLU(),
            nn.MaxPool2d(2),            # MaxPool2d는 stride 설정을 안하면 kernel_size와 같다.
        )

        self.layer2=nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.layer3=nn.Sequential(      # fully connected이다.
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )

    def forward(self,x):
        out=self.layer1(x)
        #print(out.shape)                #크기가 헷갈리면 print로 출력을 해보는 것도 방법이다.
        out=self.layer2(out)
        #print(out.shape)
        out=out.view(out.shape[0],-1)   # 펼친다. out=out.view(out.size(0),-1)로 바꿔도 된다.
        #print(out.shape)
        out=self.layer3(out)
        return out


In [11]:
# testing
net=CNN().to(device)
test_input=(torch.Tensor(3,3,64,128)).to(device)
test_out=net(test_input)

In [13]:
# optimizer & criterion
optimizer=optim.Adam(net.parameters(), lr=0.00001)
loss_func=nn.CrossEntropyLoss().to(device)

In [ ]:
total_batch=len(data_loader)

epochs=5

for epoch in range(epochs):
    avg_cost=0.0
    for num, data in enumerate(data_loader):
        imgs,labels=data
        imgs=imgs.to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        out=net(imgs)
        loss=loss_func(out, labels)
        loss.backward()
        optimizer.step()

        avg_cost+=loss/total_batch

    print('[Epoch: {}] cost={}'.format(epoch+1,avg_cost))
print('learning finished')

[Epoch: 1] cost=0.6785112619400024
[Epoch: 2] cost=0.6540599465370178
[Epoch: 3] cost=0.6283811330795288
[Epoch: 4] cost=0.5918824672698975
[Epoch: 5] cost=0.5401192307472229
learning finished


model의 save에서 다시 불러오는 방법을 간단하게 넣어두었다.

In [12]:
torch.save(net.state_dict(),'./model/model.pth')    #학습시킨 모델을 항상 쓸 때마다 다시 학습을 시켜서 사용할 수는 없으므로
                                                    #torch.save를 써서 그 network의 state_dict 값을 저장하게 된다.
                                                    #state_dict()는 torch.nn.Module에서 모델로 학습할 때 각 layer마다 텐서로 매핑되는 매개변수(예를들어, 가중치, 편향 등)
                                                    #를 python dictionary 타입으로 저장한 객체이다. 학습 가능한 매개변수를 갖는 계층만이 모델의 state_dict에 항목을 가진다.
                                                    #torch.optim 또한 옵티마이저의 상태 뿐만 아니라 사용된 하이퍼 매개변수 정보가 포함된 state_dict를 갖는다.
                                                    #
                                                    # 문법은 torch.save(모델명.state_dict(),'저장경로.pth')
                                                    # 참고로 .pth는 pytorch 모델 파일을 저장할 때 사용되는 확장자. .pth파일은 모델의 state_dict를 저장한 파일이다.

In [17]:
# 불러오려면 그 네트워크의 모형은 똑같아야 한다.
# 그래서 net을 선언할 때 했던 것처럼 똑같이 선언해 주는데, 이름은 new_net이라 하고 새로운 네트워크를 선언해주자.
new_net=CNN().to(device)

In [ ]:
new_net.load_state_dict(torch.load('./model/model.pth'))    #모델을 불러올 때는 새로운_모델명.load_state_dict()를 사용.
                                                            #torch.load를 이용해서 아까 저장한 값을 가져온다. 
                                                            #'./model/model.pth' 이 값이 아까 저장한 load_state_dict이니 load해서 new_net에 담아라는 의미다.


<All keys matched successfully>

In [ ]:
#net에서의 값과 new_net에서의 값이 일치하는 것을 관찰할 수 있다.

print(net.layer1[0])            # net의 layer1의 첫번째
print(new_net.layer1[0])        # new_net의 layer1의 첫번째

print(net.layer1[0].weight[0][0][0])    #net의 layer1의 첫번째의 weight값의 첫번째 index에 해당하는 값을 뽑아라. net의 layer1의 첫번째는 convolution layer다.
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0]==new_net.layer1[0].weight[0]        #일치.

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0087, -0.1160, -0.0399,  0.0729, -0.0699], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([-0.0087, -0.1160, -0.0399,  0.0729, -0.0699], device='cuda:0',
       grad_fn=<SelectBackward0>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]], device='cuda:0')

In [ ]:
# 학습시킨 모델들을 test해보자.
trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),                    # 64x128의 사이즈로 resize하고 
    transforms.ToTensor()                           # ToTensor로 바꿔서 사용.
])                                                  #혹시라도 trans=torchvision.transforms.Compose([transforms.ToTensor(64,128)])을 생각했다면 이건 유효한 코드가 아니다.

test_data=torchvision.datasets.ImageFolder(root='./custom_data/test_data',transform=trans)          



In [ ]:
test_set=DataLoader(dataset=test_data, batch_size=len(test_data))       # testset은 DataLoader로 해서 한번에 쓸 수 있게 test_data set의 길이만큼 batch_size로 한번에 받음.
                                                                        # 참고로 채널크기가 맞아야 하는거지 batch_size는 얼마가 들어가든 상관이 없다.
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label=data
        imgs=imgs.to(device)
        label=label.to(device)

        prediction=net(imgs)

        correct_prediction=torch.argmax(prediction,1)==label

        accuracy=correct_prediction.float().mean()
        print('Accuracy:',accuracy.item())

Accuracy: 0.7199999690055847


In [16]:
# GPU 메모리 캐시 비우기
torch.cuda.empty_cache()

# 모델과 데이터를 삭제
del net
del new_net

# 가비지 컬렉션을 강제로 호출하여 메모리 정리
import gc
gc.collect()

# Python 세션 종료
exit()


NameError: name 'net' is not defined

# 연습해보자

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

trans=torchvision.transforms.Compose([transforms.ToTensor()])
train_data=torchvision.datasets.ImageFolder(root='./custom_data/train_data',transform=trans)

data_loader=torch.utils.data.DataLoader(dataset=train_data,batch_size=8, shuffle=True,num_workers=2)


class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv2d(3,6,5),           # input_channel, output_channel, kernel_size순. stride는 기본값이 1임. padding 도 기본값이 0임.
            nn.ReLU(),
            nn.MaxPool2d(2),            # MaxPool2d는 stride 설정을 안하면 kernel_size와 같다.
        )

        self.layer2=nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.layer3=nn.Sequential(      # fully connected이다.
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )

    def forward(self,x):
        out=self.layer1(x)
        #print(out.shape)                #크기가 헷갈리면 print로 출력을 해보는 것도 방법이다.
        out=self.layer2(out)
        #print(out.shape)
        out=out.view(out.shape[0],-1)   # 펼친다. out=out.view(out.size(0),-1)로 바꿔도 된다.
        #print(out.shape)
        out=self.layer3(out)
        return out

net=CNN().to(device)

# optimizer & criterion
optimizer=optim.Adam(net.parameters(), lr=0.00001)
loss_func=nn.CrossEntropyLoss().to(device)

total_batch=len(data_loader)

epochs=5

for epoch in range(epochs):
    avg_cost=0.0
    for num,data in enumerate(data_loader):
        img,label=data
        img=img.to(device)
        label=label.to(device)
        optimizer.zero_grad()
        hypothesis=net(img)
        cost=loss_func(hypothesis,label)
        cost.backward()
        optimizer.step()
        avg_cost+=loss/total_batch

    print('[Epoch: {}] cost={}'.format(epoch+1,avg_cost))
print('learning finished')
        

In [ ]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        img, label=data
        img=img.to(device)
        label=label.to(device)
        predictions=net(img)
        correct=torch.argmax(predictions,1)==label

        accuracy=correct.float().mean()

        print(accuracy.item())

In [ ]:
exit()

: 